In [41]:
!pip install earthengine-api
!pip install geemap
!pip install geopy  # Install geopy for geocoding location names
!pip install meteostat  # Install meteostat for weather data


In [42]:
!earthengine authenticate

Authenticate: Limited support in Colab. Use ee.Authenticate() or --auth_mode=notebook instead.
Authenticate: Credentials already exist.  Use --force to refresh.


In [43]:
import ee
import geemap
import requests
import json
from geopy.geocoders import Nominatim  # Import geopy to get lat/long
from meteostat import Point, Daily
from datetime import datetime

# Initialize the Earth Engine module
ee.Initialize()

# Load the UHI dataset
dataset = ee.ImageCollection('YALE/YCEO/UHI/Summer_UHI_yearly_pixel/v4')

# Set visualization parameters
visualization = {
  'bands': ['Daytime'],
  'min': -1.5,
  'max': 7.5,
  'palette': ['313695', '74add1', 'fed976', 'feb24c', 'fd8d3c', 'fc4e2a', 'e31a1c', 'b10026']
}

# Define the location to center the map
Map = geemap.Map(center=[40.6, -74.7], zoom=7)

# Add the layer to the map
Map.addLayer(dataset, visualization, 'Daytime UHI')

# Display the map
Map


Map(center=[40.6, -74.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [52]:
# Function to get coordinates (lat, long) for each location using geopy
def get_coordinates(location_name):
    geolocator = Nominatim(user_agent="maxiattiogbe@gmail.com")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        print(f"Location not found: {location_name}")
        return None, None  # Return None if location not found

# Function to convert Celsius to Fahrenheit
def c_to_f(celsius):
    return (celsius * 9/5) + 32

# Placeholder function to calculate average day/night temperatures (replace with actual logic)
def get_average_temperatures(lat, long):
    try:
        # Define the location for Meteostat
        location = Point(lat, long)

        # Define the date ranges for summer and winter
        summer_start = datetime(2022, 6, 1)
        summer_end = datetime(2022, 8, 31)
        winter_start = datetime(2022, 12, 1)
        winter_end = datetime(2023, 2, 28)

        # Fetch daily temperature data for summer
        summer_data = Daily(location, summer_start, summer_end)
        summer_data = summer_data.fetch()

        # Fetch daily temperature data for winter
        winter_data = Daily(location, winter_start, winter_end)
        winter_data = winter_data.fetch()

        # Calculate average summer and winter temperatures in Celsius
        avg_summer_day_temp_c = summer_data['tmax'].mean()  # Max temp (day)
        avg_summer_night_temp_c = summer_data['tmin'].mean()  # Min temp (night)
        avg_winter_day_temp_c = winter_data['tmax'].mean()  # Max temp (day)
        avg_winter_night_temp_c = winter_data['tmin'].mean()  # Min temp (night)

        # Convert temperatures to Fahrenheit
        avg_summer_day_temp_f = c_to_f(avg_summer_day_temp_c)
        avg_summer_night_temp_f = c_to_f(avg_summer_night_temp_c)
        avg_winter_day_temp_f = c_to_f(avg_winter_day_temp_c)
        avg_winter_night_temp_f = c_to_f(avg_winter_night_temp_c)

        # Calculate annual averages as the average of summer and winter temperatures
        annual_day_temp_f = (avg_summer_day_temp_f + avg_winter_day_temp_f) / 2
        annual_night_temp_f = (avg_summer_night_temp_f + avg_winter_night_temp_f) / 2

        return {
            "Annual daytime SUHI": f"{annual_day_temp_f:.2f} °F",
            "Annual nighttime SUHI": f"{annual_night_temp_f:.2f} °F",
            "Summer daytime SUHI": f"{avg_summer_day_temp_f:.2f} °F",
            "Summer nighttime SUHI": f"{avg_summer_night_temp_f:.2f} °F",
            "Winter daytime SUHI": f"{avg_winter_day_temp_f:.2f} °F",
            "Winter nighttime SUHI": f"{avg_winter_night_temp_f:.2f} °F"
        }

    except Exception as e:
        print(f"Error fetching data for coordinates {lat}, {long}: {e}")
        return {
            "Annual daytime SUHI": "N/A",
            "Annual nighttime SUHI": "N/A",
            "Summer daytime SUHI": "N/A",
            "Summer nighttime SUHI": "N/A",
            "Winter daytime SUHI": "N/A",
            "Winter nighttime SUHI": "N/A"
        }

# Function to create the JSON data
def create_data_json(location_names):
    data = []

    for location_name in location_names:
        lat, long = get_coordinates(location_name)

        if lat is not None and long is not None:
            # Get averaged day/night temperatures
            temperatures = get_average_temperatures(lat, long)

            # Add location data to the JSON structure
            new_data = {
                'locationName': location_name,
                'lat': lat,
                'long': long,
            }
            for key,value in temperatures.items():
              new_data[key] = value
            data.append(new_data)

    # Save the data to a JSON file
    with open('data.json', 'w') as outfile:
        json.dump(data, outfile, indent=2)

    print("Data saved to data.json")

In [54]:
# Define a list of location names
location_names = [
    "Harvard Yard, Cambridge, MA",
    "Northeastern University Snell Library, Boston, MA",
    "Massachusetts College of Art and Design, Boston, MA",
    "Cambridge College, Cambridge, MA",
    "Fenway Park, Fenway-Kenmore, Boston, MA",
    "Back Bay Station, Boston, MA",
    "Kendall/MIT Station, Cambridge, MA",
    "Harvard Art Museums, Cambridge, MA",
    "Suffolk University 73 Tremont Street, Boston, MA",
    "Boston Public Library, Back Bay, Boston, MA",
    "Museum of Fine Arts, Boston, MA",
    "Boston Public Garden, Boston, MA",
    "MIT Stata Center, Cambridge, MA",
    "Harvard Science Center, Cambridge, MA",
    "Prudential Tower, Boston, MA",
    "Charles River Esplanade, Boston, MA",
    "Longfellow Bridge, Cambridge, MA",
    "Harvard Kennedy School, Cambridge, MA",
    "MIT Museum, Cambridge, MA",
    "Boston Common, Boston, MA",
    "Copley Square, Boston, MA",
    "Massachusetts General Hospital, Boston, MA",
    "John F. Kennedy Presidential Library, Boston, MA",
    "CambridgeSide Galleria, Cambridge, MA",
    "Newbury Street, Back Bay, Boston, MA",
    "Lechmere Canal Park, Cambridge, MA",
    "Simmons University, Boston, MA",
    "Seaport District, Boston, MA",
    "Harvard Divinity School, Cambridge, MA",
    "Harvard Law School, Cambridge, MA",
    "Boston Children's Hospital, Boston, MA"
]

# Call the function to create the data.json file
create_data_json(location_names)

Data saved to data.json
